<a href="https://qworld.net" target="_blank" align="left"><img src="./images/header.jpg"  align="left"></a>
$ \newcommand{\bra}[1]{\langle #1|} $
$ \newcommand{\ket}[1]{|#1\rangle} $
$ \newcommand{\braket}[2]{\langle #1|#2\rangle} $
$ \newcommand{\dot}[2]{ #1 \cdot #2} $
$ \newcommand{\biginner}[2]{\left\langle #1,#2\right\rangle} $
$ \newcommand{\mymatrix}[2]{\left( \begin{array}{#1} #2\end{array} \right)} $
$ \newcommand{\myvector}[1]{\mymatrix{c}{#1}} $
$ \newcommand{\myrvector}[1]{\mymatrix{r}{#1}} $
$ \newcommand{\mypar}[1]{\left( #1 \right)} $
$ \newcommand{\mybigpar}[1]{ \Big( #1 \Big)} $
$ \newcommand{\sqrttwo}{\frac{1}{\sqrt{2}}} $
$ \newcommand{\dsqrttwo}{\dfrac{1}{\sqrt{2}}} $
$ \newcommand{\onehalf}{\frac{1}{2}} $
$ \newcommand{\donehalf}{\dfrac{1}{2}} $
$ \newcommand{\hadamard}{ \mymatrix{rr}{ \sqrttwo & \sqrttwo \\ \sqrttwo & -\sqrttwo }} $
$ \newcommand{\vzero}{\myvector{1\\0}} $
$ \newcommand{\vone}{\myvector{0\\1}} $
$ \newcommand{\stateplus}{\myvector{ \sqrttwo \\  \sqrttwo } } $
$ \newcommand{\stateminus}{ \myrvector{ \sqrttwo \\ -\sqrttwo } } $
$ \newcommand{\myarray}[2]{ \begin{array}{#1}#2\end{array}} $
$ \newcommand{\X}{ \mymatrix{cc}{0 & 1 \\ 1 & 0}  } $
$ \newcommand{\I}{ \mymatrix{rr}{1 & 0 \\ 0 & 1}  } $
$ \newcommand{\Z}{ \mymatrix{rr}{1 & 0 \\ 0 & -1}  } $
$ \newcommand{\Htwo}{ \mymatrix{rrrr}{ \frac{1}{2} & \frac{1}{2} & \frac{1}{2} & \frac{1}{2} \\ \frac{1}{2} & -\frac{1}{2} & \frac{1}{2} & -\frac{1}{2} \\ \frac{1}{2} & \frac{1}{2} & -\frac{1}{2} & -\frac{1}{2} \\ \frac{1}{2} & -\frac{1}{2} & -\frac{1}{2} & \frac{1}{2} } } $
$ \newcommand{\CNOT}{ \mymatrix{cccc}{1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 0 & 1 \\ 0 & 0 & 1 & 0} } $
$ \newcommand{\norm}[1]{ \left\lVert #1 \right\rVert } $
$ \newcommand{\pstate}[1]{ \lceil \mspace{-1mu} #1 \mspace{-1.5mu} \rfloor } $
$ \newcommand{\greenbit}[1] {\mathbf{{\color{green}#1}}} $
$ \newcommand{\bluebit}[1] {\mathbf{{\color{blue}#1}}} $
$ \newcommand{\redbit}[1] {\mathbf{{\color{red}#1}}} $
$ \newcommand{\brownbit}[1] {\mathbf{{\color{brown}#1}}} $
$ \newcommand{\blackbit}[1] {\mathbf{{\color{black}#1}}} $

<font style="font-size:28px;" align="left"><b> <font color="blue"> Solutions for </font>Reflections </b></font>

_prepared by Abuzer Yakaryilmaz_

_ProjectQ adaptation by Vishal Sharathchandra Bajpe and Marija Šćekić_
<br><br>

<a id="task1"></a>
## Task 1

Design a quantum circuit with 5 qubits as follows:
- Apply H-gate (Hadamard operator) to each qubit
- Randomly pick a subset of qubits
- Apply Z-gate ($Z$ operator) to these randomly picked qubits, i.e.,


    Z | qubits[i]


- Apply H-gate to each qubit.
- Measure each qubit.

Execute your program 1000 times.

Compare the outcomes of the qubits affected by Z-gates and the outcomes of the qubits not affected by Z-gates.

Does applying Z-gate change the outcome?

If so, why?

<h3> Solution </h3>

In [ ]:
# Import all necessary libraries
from random import randrange
from projectq import MainEngine
from projectq.ops import Measure, All, H, Z, Barrier
from projectq.backends import Simulator, CircuitDrawerMatplotlib
from projectq.setups.default import get_engine_list
from projectq.meta import Loop

# Define number of qubits
number_of_qubits = 5
    
# pick a subset of {0,...,number_of_qubits-1}
picked_qubits = []
for i in range(number_of_qubits):
    if randrange(2) == 0:
        picked_qubits.append(i)

print("the indices of picked qubits are",picked_qubits)        

# create the dictionary for the outcomes
outcomes={}

# the number of iteration
iteration = 1000

# Run circuit for iteration times
for i in range (iteration):
    
    # Intialize engine objects
    qdrawer = CircuitDrawerMatplotlib()
    qengine = MainEngine(backend = Simulator(), engine_list = [qdrawer]+get_engine_list())

    
    # Define circuit paratmeters
    qubits = qengine.allocate_qureg(number_of_qubits)

    # Apply Hadamard gates at once
    All(H) | qubits

    for j in picked_qubits:
            Z | qubits[j]

    # Add Hadamard gates at once
    All(H) | qubits   

    # Barrier gate for visualization
    Barrier | qubits

    # Measure circuit
    All(Measure) | qubits
    
    qengine.flush() # send the circuit to the simulator
    
    # read the outcome of the circuit as a binary string
    key = ""
    for q in qubits: # read the outcome of each qubit
        key = key + str(int(q)) # append each outcome
    if key in outcomes: outcomes[key]+=1 # increase the frequency of key by 1
    else: outcomes[key] = 1 # create the key with value 1

# draw the circuit        
qdrawer.draw()

print("Outcomes are",outcomes)

<a id="task2"></a>
## Task 2

Randomly create a quantum state and multiply it with Hadamard matrix to find its reflection.

Draw both states.

Repeat the task for a few times.

<h3>Solution</h3>

A function for randomly creating a 2-dimensional quantum state:

In [ ]:
# randomly create a 2-dimensional quantum state
from math import cos, sin, pi
from random import randrange
def random_qstate_by_angle():
    angle_degree = randrange(360)
    angle_radian = 2*pi*angle_degree/360
    return [cos(angle_radian),sin(angle_radian)]

In [ ]:
%run quantum.py

draw_qubit()

# line of reflection for Hadamard
from matplotlib.pyplot import arrow
arrow(-1.109,-0.459,2.218,0.918,linestyle='dotted',color='red')

[x1,y1] = random_qstate_by_angle()

print(x1,y1)

sqrttwo=2**0.5
oversqrttwo = 1/sqrttwo

[x2,y2] = [ oversqrttwo*x1 + oversqrttwo*y1 , oversqrttwo*x1 - oversqrttwo*y1 ]

print(x2,y2)

draw_quantum_state(x1,y1,"main")

draw_quantum_state(x2,y2,"ref")

show_plt()

<a id="task3"></a>
<h3> Task 3 </h3>

Find the matrix representing the reflection over the line $y=x$.

<i>Hint: Think about the reflections of the points $ \myrvector{0 \\ 1} $, $ \myrvector{-1 \\ 0} $, and $ \myrvector{-\sqrttwo \\ \sqrttwo} $ over the line $y=x$.</i>

Randomly create a quantum state and multiply it with this matrix to find its reflection over the line $y = x$.

Draw both states.

Repeat the task for a few times.

<h3>Solution</h3>

The reflection over the line $y=x$ swaps the first and second amplitudes.

This is the operetor NOT: $ X = \mymatrix{rr}{0 & 1 \\ 1 & 0} $.

A function for randomly creating a 2-dimensional quantum state:

In [ ]:
# randomly create a 2-dimensional quantum state
from math import cos, sin, pi
from random import randrange
def random_qstate_by_angle():
    angle_degree = randrange(360)
    angle_radian = 2*pi*angle_degree/360
    return [cos(angle_radian),sin(angle_radian)]

Reflecting the randomly picked quantum state over the line $y=x$. 

In [ ]:
%run quantum.py

draw_qubit()

# the line y=x
from matplotlib.pyplot import arrow
arrow(-1,-1,2,2,linestyle='dotted',color='red')

[x1,y1] = random_qstate_by_angle()

[x2,y2] = [y1,x1]

draw_quantum_state(x1,y1,"main")
draw_quantum_state(x2,y2,"ref")

show_plt()